# Spitch Multilingual ASR Test (Igbo, Hausa, Yoruba)

This notebook exercises the Spitch SDK locally for:
- Speech-to-text in the source language (ig/ha/yo)
- Text translation from the source language to English

It mirrors the backend flow: client.speech.transcribe(...) then client.text.translate(...).

Prerequisites:
- Install the SDK in your venv: `pip install spitch`
- Ensure the environment variable `SPITCH_API_KEY` is set (do not hardcode secrets in notebooks).
- Provide paths to local audio files for Igbo, Hausa, and Yoruba.

In [8]:
import os, time, json
from spitch import Spitch
from dotenv import load_dotenv

load_dotenv()

# Read API key from environment; do not hardcode secrets
api_key = os.getenv('SPITCH_API_KEY')
assert api_key, 'Set SPITCH_API_KEY in your environment before running this notebook.'

# Initialize SDK client (env or explicit arg both work)
client = Spitch(api_key=api_key)
print('Spitch client initialized.')

Spitch client initialized.


In [9]:
from typing import Dict

def transcribe_and_translate(audio_path: str, lang_code: str, target: str = 'en') -> Dict:
    """
    Run Spitch ASR then translate to English.
    lang_code: 'ig' | 'ha' | 'yo'
    Returns a dict with transcript, translation, and timings.
    """
    t0 = time.time()
    with open(audio_path, 'rb') as f:
        asr = client.speech.transcribe(content=f, language=lang_code)
    t_asr = int((time.time() - t0) * 1000)
    transcript = getattr(asr, 'text', None) or getattr(asr, 'transcript', '') or ''

    t1 = time.time()
    tr = client.text.translate(text=transcript, source=lang_code, target=target)
    t_tr = int((time.time() - t1) * 1000)
    translation = getattr(tr, 'text', None) or getattr(tr, 'translation', '') or ''

    return {
        'audio_path': audio_path,
        'source_lang': lang_code,
        'target_lang': target,
        'transcript': transcript,
        'translation_en': translation,
        'timings_ms': {'asr_ms': t_asr, 'translate_ms': t_tr, 'total_ms': t_asr + t_tr},
    }

## Configure local audio file paths
Update the paths to your local audio samples for each language before running the tests below.

In [10]:
# Example paths — change to your actual files
IGBO_AUDIO = r'C:\Users\johne\Desktop\DSN\tattara\backend\ai\files\dental_exam_igbo.wav'
HAUSA_AUDIO = r'C:\Users\johne\Desktop\DSN\tattara\backend\ai\files\patient_consultation_hausa.wav'
YORUBA_AUDIO = r'C:\Users\johne\Desktop\DSN\tattara\backend\ai\files\vaccination_record_yoruba.wav'

for p in [IGBO_AUDIO, HAUSA_AUDIO, YORUBA_AUDIO]:
    print(p, 'exists:' , os.path.exists(p))

C:\Users\johne\Desktop\DSN\tattara\backend\ai\files\dental_exam_igbo.wav exists: True
C:\Users\johne\Desktop\DSN\tattara\backend\ai\files\patient_consultation_hausa.wav exists: True
C:\Users\johne\Desktop\DSN\tattara\backend\ai\files\vaccination_record_yoruba.wav exists: True


## Test Hausa (ha → en)
Runs ASR in Hausa, prints transcript, then translates to English and prints translation.

In [13]:
try:
    ha_result = transcribe_and_translate(HAUSA_AUDIO, 'ha', target='en')
    print('Hausa ASR (ha):\n', ha_result['transcript'])
    print('\nTranslation (en):\n', ha_result['translation_en'])
    print('\nTimings (ms):', ha_result['timings_ms'])
except Exception as e:
    print('Hausa test failed:', e)

Hausa ASR (ha):
 Sunana Amina Abdullahi, ni na mace ne mai shekara hamsin da biyu. An gwada ni kuma aka samu ina da cutar zazzabin cizon sauro, Malaria, tare da alamomin zazzabin cizon kai kamar ciwon kai, zafin jiki da raunin jiki an kula da ni a asibitin gwamnati na Ogidi, ana bukatar dawo don dubawa bayan kwanaki uku.

Translation (en):
 My name is Amina Abdullahi, I am a fifty-two-year-old woman. I was tested and found to have malaria, with symptoms of headache, fever, and body weakness. I was treated at Ogidi government hospital and am required to return for a check-up after three days.

Timings (ms): {'asr_ms': 131862, 'translate_ms': 3481, 'total_ms': 135343}


## Test Igbo (ig → en)
Runs ASR in Igbo, prints transcript, then translates to English and prints translation.

In [15]:
try:
    ig_result = transcribe_and_translate(IGBO_AUDIO, 'ig', target='en')
    print('Igbo ASR (ig):\n', ig_result['transcript'])
    print('\nTranslation (en):\n', ig_result['translation_en'])
    print('\nTimings (ms):', ig_result['timings_ms'])
except Exception as e:
    print('Igbo test failed:', e)

Igbo ASR (ig):
 Aha m bụ Ebuka Jọnzọn, ụbọchị enyochara m bụ nke iri abụọ nke ọktọba. Nsogbu kachasị m nwere bụ na-agba nri m n'akụ m, dọkịta kpa m, onye dọkịta eze nke lere m anya kwadoro n'eze na ọnụ m dị ọcha ma na-agba m dị m, ọ bụ ezie na-achọpụtara ohere Eze abụọ.Ụgwọ e nyere m bụ ihicha ọnụ na itinye ihe n'ohere eze. Ụbọchị nyocha ọzọ ga-abụ nke iri na itoolu nke Nọvemba.

Translation (en):
 My name is Ebuka Jonzon, the day I was examined was the twentieth of October. The main problem I have is pain in my tooth, the dentist who examined me, the dentist who looked at my tooth confirmed that my teeth and mouth are clean but I have pain, although two tooth cavities were discovered. The treatment given to me was cleaning the mouth and putting something in the cavities. The next appointment will be the nineteenth of November.

Timings (ms): {'asr_ms': 120535, 'translate_ms': 4140, 'total_ms': 124675}


## Test Yoruba (yo → en)
Runs ASR in Yoruba, prints transcript, then translates to English and prints translation.

In [18]:
try:
    yo_result = transcribe_and_translate(YORUBA_AUDIO, 'yo', target='en')
    print('Yoruba ASR (yo):\n', yo_result['transcript'])
    print('\nTranslation (en):\n', yo_result['translation_en'])
    print('\nTimings (ms):', yo_result['timings_ms'])
except Exception as e:
    print('Yoruba test failed:', e)

Yoruba ASR (yo):
 Iye aláìsàn ni a sì fún ní abé ẹ̀dẹ́fẹ́fìnì fún ààrùn àtẹ̀gùn pólíò pẹ̀lú ìgbà mẹ́ta dọ́sẹ̀ mẹ́ta àti àmúṣòrò mọ́ dáadáá, adáàbẹ̀ ẹ̀dá yìí lórí ọjọ́ kọkànlá tó ṣù kẹwàá, o sì ní láti padà wá rí dọ́kítà ní ọjọ́ kọkànlá tó dà bí ẹ̀dà náà ni dọ́kànlá tó dá abé ẹ̀dá náà ni dọkita òkè láti ilé ìwòsàn àwọn aráyé.Yaba Yaba Community ti Ospita, aláìsàn náà ni ìrírí ìṣúná Dúrúsínẹ́sì gẹ́gẹ́ bí ipa ẹgbẹ́ abẹ ẹ̀dá náà.

Translation (en):
 The patient was given intramuscular injection for polio with three doses and vaccination in addition, this dose was given on the eleventh day after birth, and the patient is to return to see the doctor on the eleventh day according to the date the dose was given by the doctor from the general hospital. The Yaba Community Hospital gave the patient emergency care as part of the dose administration.

Timings (ms): {'asr_ms': 111509, 'translate_ms': 4356, 'total_ms': 115865}


## Summary
Optionally collect results into a single structure for comparison.

In [19]:
summary = {}
if 'ha_result' in globals(): summary['ha'] = ha_result
if 'ig_result' in globals(): summary['ig'] = ig_result
if 'yo_result' in globals(): summary['yo'] = yo_result
print(json.dumps(summary, ensure_ascii=False, indent=2))

{
  "ha": {
    "audio_path": "C:\\Users\\johne\\Desktop\\DSN\\tattara\\backend\\ai\\files\\patient_consultation_hausa.wav",
    "source_lang": "ha",
    "target_lang": "en",
    "transcript": "Sunana Amina Abdullahi, ni na mace ne mai shekara hamsin da biyu. An gwada ni kuma aka samu ina da cutar zazzabin cizon sauro, Malaria, tare da alamomin zazzabin cizon kai kamar ciwon kai, zafin jiki da raunin jiki an kula da ni a asibitin gwamnati na Ogidi, ana bukatar dawo don dubawa bayan kwanaki uku.",
    "translation_en": "My name is Amina Abdullahi, I am a fifty-two-year-old woman. I was tested and found to have malaria, with symptoms of headache, fever, and body weakness. I was treated at Ogidi government hospital and am required to return for a check-up after three days.",
    "timings_ms": {
      "asr_ms": 131862,
      "translate_ms": 3481,
      "total_ms": 135343
    }
  },
  "ig": {
    "audio_path": "C:\\Users\\johne\\Desktop\\DSN\\tattara\\backend\\ai\\files\\dental_exam_igbo.wa